In [13]:
import torch
from utils.helper import ObjectView
from utils.config import config_dataset

In [14]:
args = ObjectView(config_dataset)

In [15]:
loads = torch.load("../data/Pecan Street Smart Meter Data (large) (tensor)/loads_raw.pt")
cond = torch.load("../data/Pecan Street Smart Meter Data (large) (tensor)/cond_raw.pt")
PV_base = torch.load("../data/Pecan Street Smart Meter Data (large) (tensor)/PV_base_raw.pt")


In [16]:
print(loads.shape)
print(cond.shape)
print(PV_base.shape)

torch.Size([30, 365, 96])
torch.Size([30, 365, 84])
torch.Size([7, 365, 96])


In [17]:

# Sampling
loads = loads[:args.num_class, :args.day_len, :]
cond = cond[:args.num_class, :args.day_len, :args.cond_dim]
PV_base = PV_base[:args.day_len]
# PV_base = torch.repeat_interleave(PV_base, repeats=args.num_class, dim=1)
PV_base = PV_base.permute(1, 0, 2)
X = torch.empty_like(loads)
for i in range(loads.shape[0]):
    load = loads[i]
    min_val = load.min()
    max_val = load.max()
    X[i] = 2 * (load - min_val) / (max_val - min_val) -1

print(X.shape)
print(PV_base.shape)
print(cond.shape)



torch.Size([30, 365, 96])
torch.Size([365, 7, 96])
torch.Size([30, 365, 79])


In [18]:
cond[1][0]

tensor([5.7600, 2.8800, 2.8800, 0.0000, 0.0000, 1.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 1.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 1.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000])

In [19]:
X_train = []
X_test = []
cond_train = []
cond_test = []
PV_base_train = []
PV_base_test = []
for i in range(args.num_class):
    indices = torch.randperm(args.day_len)
    train_size = int(args.train_ratio * args.day_len)
    test_size = args.day_len - train_size
    train_indices, test_indices = indices[:train_size], indices[train_size:]
    X_train.append(X[i][train_indices])
    X_test.append(X[i][test_indices])
    cond_train.append(cond[i][train_indices])
    cond_test.append(cond[i][test_indices])
    PV_base_train.append(PV_base[train_indices])
    PV_base_test.append(PV_base[test_indices])

X_train = torch.stack(X_train)
X_test = torch.stack(X_test)
cond_train = torch.stack(cond_train)
cond_test = torch.stack(cond_test)
PV_base_train = torch.stack(PV_base_train)
PV_base_test = torch.stack(PV_base_test)


In [20]:
X_train.shape, X_test.shape, cond_train.shape, cond_test.shape, PV_base_train.shape, PV_base_test.shape

(torch.Size([30, 292, 96]),
 torch.Size([30, 73, 96]),
 torch.Size([30, 292, 79]),
 torch.Size([30, 73, 79]),
 torch.Size([30, 292, 7, 96]),
 torch.Size([30, 73, 7, 96]))

In [21]:

X_train = X_train.reshape(-1, X_train.shape[-1])
X_test = X_test.reshape(-1, X_test.shape[-1])
cond_train = cond_train.reshape(-1, cond_train.shape[-1])
cond_test = cond_test.reshape(-1, cond_test.shape[-1])
PV_base_train = PV_base_train.reshape(-1, PV_base_train.shape[-2], PV_base_train.shape[-1])
PV_base_test = PV_base_test.reshape(-1, PV_base_train.shape[-2], PV_base_test.shape[-1])

In [22]:
X_train.shape, X_test.shape, cond_train.shape, cond_test.shape, PV_base_train.shape, PV_base_test.shape

(torch.Size([8760, 96]),
 torch.Size([2190, 96]),
 torch.Size([8760, 79]),
 torch.Size([2190, 79]),
 torch.Size([8760, 7, 96]),
 torch.Size([2190, 7, 96]))

In [23]:
num_class = config_dataset["num_class"]

def denorm(x):
    loads = torch.load("../data/Pecan Street Smart Meter Data (large) (tensor)/loads_raw.pt")
    lmin = []
    lmax = []
    for i in range(loads.shape[0]):
        load = loads[i]
        lmin.append(load.min())
        lmax.append(load.max())
    lmin = torch.stack(lmin)[:num_class]
    lmax = torch.stack(lmax)[:num_class]
    lmin = lmin.unsqueeze(1).repeat(1, int(x.shape[0]/num_class))
    lmax = lmax.unsqueeze(1).repeat(1, int(x.shape[0]/num_class))
    lmin = lmin.reshape((-1, 1))
    lmax = lmax.reshape((-1, 1))
    return (x + 1) * (lmax - lmin)/ 2 + lmin

In [24]:
torch.save(X_train, "../data/Pecan Street Smart Meter Data (large) (tensor)/X_train.pt")
torch.save(X_test, "../data/Pecan Street Smart Meter Data (large) (tensor)/X_test.pt")
torch.save(cond_train, "../data/Pecan Street Smart Meter Data (large) (tensor)/cond_train.pt")
torch.save(cond_test, "../data/Pecan Street Smart Meter Data (large) (tensor)/cond_test.pt")
torch.save(PV_base_train, "../data/Pecan Street Smart Meter Data (large) (tensor)/PV_base_train.pt")
torch.save(PV_base_test, "../data/Pecan Street Smart Meter Data (large) (tensor)/PV_base_test.pt")
torch.save(denorm(X_test), "../data/Pecan Street Smart Meter Data (large) (tensor)/load_test.pt")
torch.save(denorm(X_train), "../data/Pecan Street Smart Meter Data (large) (tensor)/load_train.pt")